In [ ]:
!/usr/local/cuda/bin/nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cuda-install-samples-11.2.sh ~ && cd /root/NVIDIA_CUDA-11.2_Samples/

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-oz_zrjqu
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-oz_zrjqu
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=824ba3b6fad041d013ff51de9664907a1c640ab2ca4acaa223a8734575b11f0f
  Stored in directory: /tmp/pip-ephem-wheel-cache-kkpbfy67/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
cr

In [ ]:
%%cuda --name curand.cu
#include <time.h>
#include <curand.h>
#include <cublas_v2.h>
#include <curand_kernel.h>
#include <iostream>
#pragma comment (lib, "cublas.lib")
#pragma comment (lib, "curand.lib")


#define IDX2C(i,j,ld) (((j)*(ld))+(i))

// Функция случайной генерации матрицы на GPU
void GPU_fill_rand(float* A, int nr_rows_A, int nr_cols_A) {
    // Create a pseudo-random number generator
    curandGenerator_t prng;
    curandCreateGenerator(&prng, CURAND_RNG_PSEUDO_DEFAULT);
    
    // Set the seed for the random number generator using the system clock
    curandSetPseudoRandomGeneratorSeed(prng, (unsigned long long) clock());

    // Fill the array with random numbers on the device
    curandGenerateUniform(prng, A, nr_rows_A * nr_cols_A);
}


// Multiply the arrays A and B on GPU and save the result in C
// C(m,n) = A(m,k) * B(k,n)
void gpu_blas_mmul(const float* A, const float* B, float* C, const int m, const int k, const int n) {
    int lda = m, ldb = k, ldc = n;
    const float alf = 1;
    const float bet = 0;
    const float* alpha = &alf;
    const float* beta = &bet;
    
    // Create a handle for CUBLAS
    cublasHandle_t handle;
    cublasCreate(&handle);
    
    // Do the actual multiplication
    cublasSgemm(handle, CUBLAS_OP_T, CUBLAS_OP_T, m, n, k, alpha, A, lda, B, ldb, beta, C, ldc);
    //(cublasHandle_t handle, cublasOperation_t transa, cublasOperation_t transb, int m, int n, int k, const TYPE *alpha, 
    // const TYPE *A, int lda, const TYPE *B, int ldb, const TYPE *beta, TYPE *C, int ldc);
    
    // Destroy the handle
    cublasDestroy(handle);
}

//Функция для вывода матрицы
void print_matrix(float* matrix, int rows, int cols) {
    for (int i = 0; i < rows; ++i) {
        for (int j = 0; j < cols; ++j) {
            printf("%f ", matrix[j * rows + i]);
        }
        printf("\n");
    }
    printf("\n");

}


//Последовательная программа
// C(m,n) = A(m,k) * B(k,n)
void consistent(const float* A, const float* B, float*C, const int m, const int k, const int n) {
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            C[IDX2C(i, j, n)] = 0.0;
            for (int r = 0; r < n; ++r) {
                C[IDX2C(i, j, n)] += A[IDX2C(i, r, k)] * B[IDX2C(r, j, n)];
            }
        }
    }
}


int main() {
    // Allocate 3 arrays on CPU
    int nr_rows_A, nr_cols_A, nr_rows_B, nr_cols_B, nr_rows_C, nr_cols_C;
    // for simplicity we are going to use square arrays
    nr_rows_A = nr_cols_A = nr_rows_B = nr_cols_B = nr_rows_C = nr_cols_C = 256;
    
    float* h_A = (float*)malloc(nr_rows_A * nr_cols_A * sizeof(float));
    float* h_B = (float*)malloc(nr_rows_B * nr_cols_B * sizeof(float));
    float* h_C = (float*)malloc(nr_rows_C * nr_cols_C * sizeof(float));
    printf("Var8\n");
    double begin = clock();
    
    for (int i = 0; i < nr_rows_A * nr_rows_A; i++) {
		h_A[i] = (float)rand();
		h_B[i] = (float)rand();
	}
  
    // Allocate 3 arrays on GPU
    float* d_A, * d_B, * d_C;
    

    cudaMalloc(&d_A, nr_rows_A * nr_cols_A * sizeof(float));
    cudaMalloc(&d_B, nr_rows_B * nr_cols_B * sizeof(float));
    cudaMalloc(&d_C, nr_rows_C * nr_cols_C * sizeof(float));
       

    // Optionally we can copy the data back on CPU and print the arrays
  cudaMemcpy(d_A, h_A, nr_rows_A * nr_cols_A*sizeof(float), cudaMemcpyHostToDevice);
	cudaMemcpy(d_B, h_B, nr_rows_B * nr_cols_B*sizeof(float), cudaMemcpyHostToDevice);
    

// Multiply A and B on GPU
    //double time_g = 0.0;
    //    srand(time(0));
    double begin1 = clock();
    gpu_blas_mmul(d_A, d_B, d_C, nr_rows_A, nr_cols_A, nr_cols_B);
    double end = (clock() - begin) / CLOCKS_PER_SEC;
    printf("\nTime1 : %f\n", end);
    
    
    //последовательная:
    double time_c = 0.0;
    //for (int step = 0; step < 12; ++step) {
        srand(time(0));
        double begin2 = clock();
        consistent(h_A, h_B, h_C, nr_rows_A, nr_cols_A, nr_cols_B);
        double end2 = double(clock() - begin2) / CLOCKS_PER_SEC;
        time_c += end2;
    //}
    
    printf("\nTime of consistent programm: %f\n", time_c);
    //std::cout << "C by consistent programm:" << std::endl;
    //print_matrix(h_C, nr_rows_C, nr_cols_C);
    // Copy (and print) the result on host memory
    //cudaMemcpy(h_C, d_C, nr_rows_C * nr_cols_C * sizeof(float), cudaMemcpyDeviceToHost);
    //printf("\nTime of CUDA programm: %f\n", end);
    //std::cout << "C by CUDA program:" << std::endl;
    //print_matrix(h_C, nr_rows_C, nr_cols_C);

    //Free GPU memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free CPU memory
    free(h_A);
    free(h_B);
    free(h_C);
    return 0;
}


'File written in /content/src/curand.cu'

In [ ]:

!nvcc -o /content/src/curand /content/src/curand.cu -lcurand -lcublas

In [ ]:
!/content/src/curand


Var8

Time1 : 1.132654

Time of consistent programm: 0.085006


In [ ]:
%%cuda --name curand1.cu
#include <time.h>
#include <curand.h>
#include <cublas_v2.h>
#include <curand_kernel.h>
#include <iostream>
#pragma comment (lib, "cublas.lib")
#pragma comment (lib, "curand.lib")


#define IDX2C(i,j,ld) (((j)*(ld))+(i))

// Функция случайной генерации матрицы на GPU
void GPU_fill_rand(float* A, int nr_rows_A, int nr_cols_A) {
    // Create a pseudo-random number generator
    curandGenerator_t prng;
    curandCreateGenerator(&prng, CURAND_RNG_PSEUDO_DEFAULT);
    
    // Set the seed for the random number generator using the system clock
    curandSetPseudoRandomGeneratorSeed(prng, (unsigned long long) clock());

    // Fill the array with random numbers on the device
    curandGenerateUniform(prng, A, nr_rows_A * nr_cols_A);
}


// Multiply the arrays A and B on GPU and save the result in C
// C(m,n) = A(m,k) * B(k,n)
void gpu_blas_mmul(const float* A, const float* B, float* C, const int m, const int k, const int n) {
    int lda = m, ldb = k, ldc = n;
    const float alf = 1;
    const float bet = 0;
    const float* alpha = &alf;
    const float* beta = &bet;
    
    // Create a handle for CUBLAS
    cublasHandle_t handle;
    cublasCreate(&handle);
    
    // Do the actual multiplication
    cublasSgemm(handle, CUBLAS_OP_T, CUBLAS_OP_T, m, n, k, alpha, A, lda, B, ldb, beta, C, ldc);
    //(cublasHandle_t handle, cublasOperation_t transa, cublasOperation_t transb, int m, int n, int k, const TYPE *alpha, 
    // const TYPE *A, int lda, const TYPE *B, int ldb, const TYPE *beta, TYPE *C, int ldc);
    
    // Destroy the handle
    cublasDestroy(handle);
}

//Функция для вывода матрицы
void print_matrix(float* matrix, int rows, int cols) {
    for (int i = 0; i < rows; ++i) {
        for (int j = 0; j < cols; ++j) {
            printf("%f ", matrix[j * rows + i]);
        }
        printf("\n");
    }
    printf("\n");

}


//Последовательная программа
// C(m,n) = A(m,k) * B(k,n)
void consistent(const float* A, const float* B, float*C, const int m, const int k, const int n) {
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            C[IDX2C(i, j, n)] = 0.0;
            for (int r = 0; r < n; ++r) {
                C[IDX2C(i, j, n)] += A[IDX2C(i, r, k)] * B[IDX2C(r, j, n)];
            }
        }
    }
}


//float, (256, 1024, 4096), (с Т, c Т) 
//тип,размерность матриц (матрицы квадратные), показатель транспонирования матриц.
int main() {
    
    printf("Вариант 8\n");
    // Allocate 3 arrays on CPU
    int nr_rows_A, nr_cols_A, nr_rows_B, nr_cols_B, nr_rows_C, nr_cols_C;
    
    // for simplicity we are going to use square arrays
    //nr_rows_A = nr_cols_A =nr_rows_B = nr_cols_B=nr_rows_C = nr_cols_C= 256;
    //nr_rows_A = nr_cols_A=nr_rows_B = nr_cols_B =nr_rows_C = nr_cols_C= 1024;
    nr_rows_A = nr_cols_A=nr_rows_B = nr_cols_B=nr_rows_C = nr_cols_C = 4096;
    
    float* h_A = (float*)malloc(nr_rows_A * nr_cols_A * sizeof(float));
    float* h_B = (float*)malloc(nr_rows_B * nr_cols_B * sizeof(float));
    float* h_C = (float*)malloc(nr_rows_C * nr_cols_C * sizeof(float));
    

    // Allocate 3 arrays on GPU
    float* d_A, * d_B, * d_C;
    cudaMalloc(&d_A, nr_rows_A * nr_cols_A * sizeof(float));
    cudaMalloc(&d_B, nr_rows_B * nr_cols_B * sizeof(float));
    cudaMalloc(&d_C, nr_rows_C * nr_cols_C * sizeof(float));
    
      

    // Multiply A and B on GPU
    double time_g = 0.0;
    double begin = clock();
    GPU_fill_rand(d_A, nr_rows_A, nr_cols_A);
    GPU_fill_rand(d_B, nr_rows_B, nr_cols_B);
   
    // Optionally we can copy the data back on CPU and print the arrays
    cudaMemcpy(h_A, d_A, nr_rows_A * nr_cols_A * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_B, d_B, nr_rows_B * nr_cols_B * sizeof(float), cudaMemcpyDeviceToHost);

    //cudaMemcpy(d_A, h_A, nr_rows_A * nr_cols_A * sizeof(float), cudaMemcpyDeviceToHost);
    //cudaMemcpy(d_B, h_B, nr_rows_B * nr_cols_B * sizeof(float), cudaMemcpyDeviceToHost);


    
    gpu_blas_mmul(d_A, d_B, d_C, nr_rows_A, nr_cols_A, nr_cols_B);
    double end = (clock() - begin) / CLOCKS_PER_SEC;
    //time_g += end;
    //}
    
    // Copy (and print) the result on host memory
    cudaMemcpy(h_C, d_C, nr_rows_C * nr_cols_C * sizeof(float), cudaMemcpyDeviceToHost);
    
    printf("\nTime of CUDA programm: %f\n", end);
    
    //std::cout << "C by CUDA program:" << std::endl;
    //print_matrix(h_C, nr_rows_C, nr_cols_C);


    int T=12;
    //последовательная реализация
    /*double time_c = 0.0;
    for (int t = 0; t < T; t++) {
        srand(time(0));
        double begin = clock();
        consistent(h_A, h_B, h_C, nr_rows_A, nr_cols_A, nr_cols_B);
        double end = double(clock() - begin) / CLOCKS_PER_SEC;
        time_c += end;
    }
    printf("\nTime of consistent programm: %f\n", time_c / T);*/
    //std::cout << "C by consistent programm:" << std::endl;
    //print_matrix(h_C, nr_rows_C, nr_cols_C);


    //Free GPU memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free CPU memory
    free(h_A);
    free(h_B);
    free(h_C);


    return 0;
}

'File written in /content/src/curand1.cu'

In [ ]:
!nvcc -o /content/src/curand1 /content/src/curand1.cu -lcurand -lcublas

/content/src/curand1.cu(102): warning: variable "time_g" was declared but never referenced

/content/src/curand1.cu(130): warning: variable "T" was declared but never referenced



In [ ]:
!/content/src/curand1

Вариант 8

Time of CUDA programm: 0.669311
